# Frida检测与反检测


## 一、Frida检测
　　常见的Frida检测
1. 检测frida-server文件名  
2. 检测27042默认端口  
3. 检测D-Bus  
4. 检测/proc/pid/maps映射文件  
5. 检测/proc/pid/tast/tid/stat或/proc/pid/tast/tid/status    
6. 双进程保护  

　　前两种可以通过修改frida-server文件名，修改默认端口绕过。双进程可以通过-f spawn模式启动绕过。其他的需要去hook修改。 
　　

## 二、Frida反检测

### 2.1、检测点定位
　　1、修改了frida-server文件名，以及服务端口，或尝试spawn启动，判定是否可以绕过检测。
　　2、so文件查看：看看载入了那些so文件？
```js
function fridaProcess(){
  Java.perform(function () {
    var enumMoudle = Process.enumerateModules();
    for (var i = 0; i < enumMoudle.length; i++){
      console.log("", enumMoudle[i].name)
    }
  });
}

setImmediate(fridaProcess,0)
```
　　3、so载入的时候，底层最后是open去打开的。所以再用frida去hook应用中的open函数，看看读取了哪些so或者文件，可以看到最后断在了/proc/self/maps（安全检测了/proc/pid/maps映射文件 ）。  
```js

var pth = Module.findExportByName(null,"open");
    Interceptor.attach(ptr(pth),{
        onEnter:function(args){
            this.filename = args[0];
            console.log("",this.filename.readCString())
            if (this.filename.readCString().indexOf(".so") != -1){
                args[0] = ptr(0)

            }

        },onLeave:function(retval){
            return retval;
        }
    })
```
### 2.2、/proc/pid/maps检测的原理分析与绕过
#### 2.2.1、原理
　　当挂上frida后，对应的maps文件中会出现re.frida.server之类的特征，这是在使用frida server的时候自动创建的，其中存放着frida的功能模块，可以在载入so的hook脚本输出中能看到最后也是断在frida-agent.so。  
#### 2.2.2、绕过
　　要绕过这个检测，可以通过备份一个正常启动时的maps文件（前提是：app不使用frida是能正常启动不闪退的）。 
```js
function main() {
  const openPtr = Module.getExportByName('libc.so', 'open');
  const open = new NativeFunction(openPtr, 'int', ['pointer', 'int']);
  var readPtr = Module.findExportByName("libc.so", "read");
  var read = new NativeFunction(readPtr, 'int', ['int', 'pointer', "int"]);
  var fakePath = "/data/data/com.app/maps";
  var file = new File(fakePath, "w");
  var buffer = Memory.alloc(512);
  Interceptor.replace(openPtr, new NativeCallback(function (pathnameptr, flag) {
      var pathname = Memory.readUtf8String(pathnameptr);
      var realFd = open(pathnameptr, flag);
      if (pathname.indexOf("maps") >= 0) {
          while (parseInt(read(realFd, buffer, 512)) !== 0) {
              var oneLine = Memory.readCString(buffer);
              if (oneLine.indexOf("tmp") === -1) {
                  file.write(oneLine);
              }
          }
          var filename = Memory.allocUtf8String(fakePath);
          return open(filename, flag);
      }
      var fd = open(pathnameptr, flag);
      return fd;
  }, 'int', ['pointer', 'int']));
}
setImmediate(main)
```